In [14]:
import os
import time
import random
import numpy as np
import pandas as pd
from fastprogress import master_bar, progress_bar

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from  torch.utils.data import DataLoader, Dataset
import torch.optim.lr_scheduler as lr_scheduler
# from CenterLoss import CenterLoss
from torch.autograd.function import Function

from sklearn.model_selection import KFold
from sklearn.metrics import recall_score

%matplotlib inline
import matplotlib.pyplot as plt

In [15]:
SEED = 2020
EPOCH = 50
IMG_SIZE = 28
NUM_CLASSES = 10
BATCH_SIZE = 128

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(SEED)

In [17]:
class MnistDataset(Dataset):
    
    def __init__(self, df, n_channels=1, is_train=True, transforms=None):
        self.data = df.iloc[:, 1:].values
#         self.fnames = df['image_id'].values
        self.n_channels = n_channels
        self.labels = df['label'].values
        self.transforms = transforms
        self.is_train = is_train
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx, :].reshape(IMG_SIZE, IMG_SIZE).astype(np.uint8)
        image = (image*(255.0/image.max())).astype(np.uint8)
        
        if self.transforms:
            if self.transforms.albumentations:
                aug = Augmentation().get_augmentation(self.transforms.albumentations)
                augmented = aug(image=image)
                image = augmented['image'].astype(np.float32)

        image = image.reshape(1, IMG_SIZE, IMG_SIZE).astype(np.float32)
        if self.n_channels > 1:
            image = np.concatenate([image for i in range(self.n_channels)], axis=0)

        if self.is_train:
            label = self.labels[idx]
            return image, label
        else:
            return image

In [18]:
train_df = pd.read_csv('../data/input/train.csv')

In [19]:
folds = pd.DataFrame(index=train_df.index.values)
folds['fold_0'] = 0

fold = KFold(n_splits=5, shuffle=True, random_state=SEED)
for fold_, (trn_idx, val_idx) in enumerate(fold.split(train_df)):
    folds.loc[val_idx, f'fold_{fold_}'] = 0.2
    folds.loc[trn_idx, f'fold_{fold_}'] = 0

In [20]:
fold_num = 0
x_trn = train_df[folds[f'fold_{fold_num}'] > 0]
x_val = train_df[folds[f'fold_{fold_num}'] == 0]

y_val = x_val['label']

In [21]:
train_dataset = MnistDataset(x_trn, n_channels=1, transforms=None)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

valid_dataset = MnistDataset(x_val, n_channels=1, transforms=None)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

### source: https://github.com/ronghuaiyang/arcface-pytorch/blob/master/models/metrics.py

In [22]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math


class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output


class AddMarginProduct(nn.Module):
    r"""Implement of large margin cosine distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        s: norm of input feature
        m: margin
        cos(theta) - m
    """

    def __init__(self, in_features, out_features, s=30.0, m=0.40):
        super(AddMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        phi = cosine - self.m
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device='cuda')
        # one_hot = one_hot.cuda() if cosine.is_cuda else one_hot
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + 'in_features=' + str(self.in_features) \
               + ', out_features=' + str(self.out_features) \
               + ', s=' + str(self.s) \
               + ', m=' + str(self.m) + ')'


class SphereProduct(nn.Module):
    r"""Implement of large margin cosine distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        m: margin
        cos(m*theta)
    """
    def __init__(self, in_features, out_features, m=4):
        super(SphereProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        self.base = 1000.0
        self.gamma = 0.12
        self.power = 1
        self.LambdaMin = 5.0
        self.iter = 0
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform(self.weight)

        # duplication formula
        self.mlambda = [
            lambda x: x ** 0,
            lambda x: x ** 1,
            lambda x: 2 * x ** 2 - 1,
            lambda x: 4 * x ** 3 - 3 * x,
            lambda x: 8 * x ** 4 - 8 * x ** 2 + 1,
            lambda x: 16 * x ** 5 - 20 * x ** 3 + 5 * x
        ]

    def forward(self, input, label):
        # lambda = max(lambda_min,base*(1+gamma*iteration)^(-power))
        self.iter += 1
        self.lamb = max(self.LambdaMin, self.base * (1 + self.gamma * self.iter) ** (-1 * self.power))

        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cos_theta = F.linear(F.normalize(input), F.normalize(self.weight))
        cos_theta = cos_theta.clamp(-1, 1)
        cos_m_theta = self.mlambda[self.m](cos_theta)
        theta = cos_theta.data.acos()
        k = (self.m * theta / 3.14159265).floor()
        phi_theta = ((-1.0) ** k) * cos_m_theta - 2 * k
        NormOfFeature = torch.norm(input, 2, 1)

        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cos_theta.size())
        one_hot = one_hot.cuda() if cos_theta.is_cuda else one_hot
        one_hot.scatter_(1, label.view(-1, 1), 1)

        # --------------------------- Calculate output ---------------------------
        output = (one_hot * (phi_theta - cos_theta) / (1 + self.lamb)) + cos_theta
        output *= NormOfFeature.view(-1, 1)

        return output

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + 'in_features=' + str(self.in_features) \
               + ', out_features=' + str(self.out_features) \
               + ', m=' + str(self.m) + ')'

In [30]:
# Model
model = models.resnet18().to(device)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
model.fc = nn.Linear(in_features=512, out_features=512, bias=True).to(device)

metric_fc = ArcMarginProduct(512, NUM_CLASSES, s=30, m=0.5, easy_margin=False).to(device)

In [31]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
sheduler = lr_scheduler.StepLR(optimizer, 20, gamma=0.8)

In [32]:
best_epoch = -1
best_val_score = -np.inf
mb = master_bar(range(EPOCH))

train_loss_list = []
val_loss_list = []
val_score_list = []

counter = 0

for epoch in mb:
    start_time = time.time()
    model.train()
    avg_loss = 0.

    for images, labels in progress_bar(train_loader, parent=mb):
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        
        feature = model(images.float())
        preds = metric_fc(feature, labels)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    train_loss_list.append(avg_loss)

    

    model.eval()
    valid_preds = np.zeros((len(valid_loader.dataset), NUM_CLASSES))
    avg_val_loss = 0.

    for i, (images, labels) in enumerate(valid_loader):
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        feature = model(images.float())
        preds = metric_fc(feature, labels)
        loss = criterion(preds, labels)

        valid_preds[i * BATCH_SIZE: (i + 1) * BATCH_SIZE] = preds.cpu().detach().numpy()
        avg_val_loss += loss.item() / len(valid_loader)

    val_score = recall_score(y_val, np.argmax(valid_preds, axis=1), average='macro')

    val_loss_list.append(avg_val_loss)
    val_score_list.append(val_score)
        
    elapsed = time.time() - start_time
    mb.write(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f} val_score: {val_score:.4f} time: {elapsed:.0f}s')

    if best_val_score < val_score:
        best_epoch = epoch + 1
        best_val_score = val_score
        best_valid_preds = valid_preds
        best_model = model.state_dict()
        counter = 0
        
    counter += 1
    if counter == 100:
        break

print('\n\n===================================\n')
print(f'CV: {best_val_score}\n')
print(f'BEST EPOCH: {best_epoch}')
print(f'BEST RECALL: {best_val_score}')
print('\n===================================\n\n')




CV: 0.9653062251567915

BEST EPOCH: 40
BEST RECALL: 0.9653062251567915



